In [2]:
import gurobipy as gp
from gurobipy import GRB

# Define the size of the Sudoku grid
grid = 9
subgrid = 3

# Create a Gurobi model
model = gp.Model("Sudoku")

# Define decision variables x[v, r, c]
x = {}
for v in range(grid):
    for r in range(grid):
        for c in range(grid):
            x[v, r, c] = model.addVar(vtype=GRB.BINARY, name=f'x[{v},{r},{c}]')

# Set the objective to minimize 0 (This is not necessary for Sudoku solving)
model.setObjective(0, GRB.MINIMIZE)

# Add constraint: Each cell must contain exactly one value
for r in range(grid):
    for c in range(grid):
        model.addConstr(gp.quicksum(x[v, r, c] for v in range(grid)) == 1)

# Add constraint: Each value must appear exactly once in each row
for v in range(grid):
    for r in range(grid):
        model.addConstr(gp.quicksum(x[v, r, c] for c in range(grid)) == 1)

# Add constraint: Each value must appear exactly once in each column
for v in range(grid):
    for c in range(grid):
        model.addConstr(gp.quicksum(x[v, r, c] for r in range(grid)) == 1)

# Add constraint: Each value must appear exactly once in each subgrid
for v in range(grid):
    for p in range(subgrid):
        for q in range(subgrid):
            r_range = range(3 * p, 3 * (p + 1))
            c_range = range(3 * q, 3 * (q + 1))
            model.addConstr(gp.quicksum(x[v, r, c] for r in r_range for c in c_range) == 1)

# Define the initial values
initial_values = {
    (5, 0, 2), (7, 0, 4), (4, 0, 6), (8, 1, 0), (2, 1, 3), (1, 1, 7), (3, 2, 1),
    (0, 2, 5), (6, 2, 8), (4, 3, 5), (5, 3, 6), (1, 4, 1), (3, 4, 4), (0, 4, 7),
    (2, 5, 0), (8, 5, 8), (0, 6, 2), (8, 6, 3), (2, 6, 7), (1, 7, 4), (3, 7, 6),
    (4, 8, 1), (6, 8, 5)
}

# Add constraint: Set the initial values
for v, r, c in initial_values:
    model.addConstr(x[v, r, c] == 1)

# Optimize the model
model.optimize()

# Check the optimization status
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found!")
    print("Objective Value:", model.objVal)

    # Prepare to print the initial Sudoku grid
    initial_grid = [['.' for _ in range(grid)] for _ in range(grid)]
    for v, r, c in initial_values:
        initial_grid[r][c] = str(v + 1)

    print("\nInitial Sudoku Grid:")
    for row in initial_grid:
        print(" ".join(row))

    # Prepare to print the solved Sudoku grid
    solved_grid = [['.' for _ in range(grid)] for _ in range(grid)]
    for r in range(grid):
        for c in range(grid):
            for v in range(grid):
                if x[v, r, c].x > 0.5:  # Check if the variable is 1 (approximately)
                    solved_grid[r][c] = str(v + 1)

    print("\nSolved Sudoku Grid:")
    for row in solved_grid:
        print(" ".join(row))
else:
    print("No optimal solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-13
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 347 rows, 729 columns and 2939 nonzeros
Model fingerprint: 0x6df3b497
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 347 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Optimal So